In [6]:
import pandas as pd
import numpy as np
ventas_df = pd.read_csv("/lakehouse/default/Files/dunnhumby - The Complete Journey CSV/transaction_data.csv")
clientes_df = pd.read_csv("/lakehouse/default/Files/dunnhumby - The Complete Journey CSV/hh_demographic.csv")

StatementMeta(, 65b88c1e-dcb6-4791-9ec1-ab1342da0c11, 8, Finished, Available)

In [3]:
#Hacemos un cálculo rápido de Recencia, Frecuencia y valor Monetario de nuestros clientes. Aquellos clientes que hace más de 600 días que no nos compran los consideramos perdidos

df_RFM =ventas_df.groupby(by="household_key").agg({'SALES_VALUE':'sum' , 'BASKET_ID':'nunique', 'DAY':'max'}).sort_values(by='SALES_VALUE',ascending=False)

df_RFM.rename(columns={'DAY': 'Recency',
                         'BASKET_ID': 'Frequency',
                         'SALES_VALUE': 'MonetaryValue'}, inplace=True)



df_lostcustomers = df_RFM[df_RFM['Recency'] < 600]
df_lostcustomers.head()

StatementMeta(, 65b88c1e-dcb6-4791-9ec1-ab1342da0c11, 5, Finished, Available)

,MonetaryValue,Frequency,Recency
household_key,,,
1828,5813.77,86,570
1974,5266.00,198,510
2051,4923.16,288,462
682,4438.29,217,569
1031,4232.12,146,557


In [10]:
#Nos quedamos con todos los clientes que haya comprado alguna vez, y con sus datos demográficos

join_customers = clientes_df.join(df_lostcustomers , on='household_key' , how='left')

join_customers['Churn'] = np.isnan(join_customers['MonetaryValue']) #True are customer who stay

caracteristicas =['household_key','AGE_DESC' , 'MARITAL_STATUS_CODE' , 'INCOME_DESC' , 'HOMEOWNER_DESC' , 'HH_COMP_DESC' , 'HOUSEHOLD_SIZE_DESC' , 'KID_CATEGORY_DESC']
etiqueta = ['Churn']

StatementMeta(, 65b88c1e-dcb6-4791-9ec1-ab1342da0c11, 12, Finished, Available)

In [11]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(join_customers[caracteristicas], join_customers[etiqueta], random_state=0)

StatementMeta(, 65b88c1e-dcb6-4791-9ec1-ab1342da0c11, 13, Finished, Available)

In [14]:
# Import the AutoML class from the FLAML package
from flaml import AutoML
from flaml.automl.spark.utils import to_pandas_on_spark

# Create an AutoML instance
automl = AutoML()

StatementMeta(, 65b88c1e-dcb6-4791-9ec1-ab1342da0c11, 16, Finished, Available)

In [15]:
# Define AutoML settings
settings = {
    "time_budget": 250,         # Total running time in seconds
    "metric": 'roc_auc',       # Optimization metric (ROC AUC in this case)
    "task": 'classification',  # Task type (classification)
    "log_file_name": 'flaml_experiment.log',  # FLAML log file
    "seed": 41,                # Random seed
    "force_cancel": True,      # Force stop training once time_budget is used up
    "mlflow_exp_name": "automl_sample"      # MLflow experiment name
}

StatementMeta(, 65b88c1e-dcb6-4791-9ec1-ab1342da0c11, 17, Finished, Available)

In [17]:
'''The main flaml automl API'''

with mlflow.start_run(nested=True):
    automl.fit(dataframe=join_customers, label='Churn', isUnbalance=True, **settings)

StatementMeta(, 65b88c1e-dcb6-4791-9ec1-ab1342da0c11, 19, Finished, Available)

[flaml.automl.logger: 06-15 08:05:33] {1766} INFO - task = classification
[flaml.automl.logger: 06-15 08:05:33] {1777} INFO - Evaluation method: cv
class 0 augmented from 6 to 24
[flaml.automl.logger: 06-15 08:05:33] {1875} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 06-15 08:05:35] {1993} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'catboost', 'lrl1']
[flaml.automl.logger: 06-15 08:05:35] {2303} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 06-15 08:05:35] {2515} INFO - stop trying learner lgbm
[flaml.automl.logger: 06-15 08:05:35] {2303} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 06-15 08:05:35] {2515} INFO - stop trying learner lgbm
[flaml.automl.logger: 06-15 08:05:35] {2303} INFO - iteration 2, current learner lgbm
[flaml.automl.logger: 06-15 08:05:35] {2515} INFO - stop trying learner lgbm
[flaml.automl.logger: 06-15 08:05:35] {2303} INFO - iteration 3, current 

In [ ]:
import flaml.visualization as fviz
fig = fviz.plot_optimization_history(automl)
# or
#fig = fviz.plot(automl, "optimization_history")
fig.show()

In [ ]:
# Retrieve and display the best hyperparameter configuration and metrics
print('Best hyperparameter config:', automl.best_config)
print('Best ROC AUC on validation data: {0:.4g}'.format(1 - automl.best_loss))
print('Training duration of the best run: {0:.4g} s'.format(automl.best_config_train_time))

In [ ]:
# Si necesitamos paralelizar
settings = {
    "time_budget": 250,           # Total running time in seconds
    "metric": 'roc_auc',         # Optimization metric (ROC AUC in this case)
    "task": 'classification',    # Task type (classification)
    "seed": 41,                  # Random seed
    "use_spark": True,           # Enable Spark-based parallelism
    "n_concurrent_trials": 3,    # Number of concurrent trials to run
    "force_cancel": True,        # Force stop training once time_budget is used up
    "mlflow_exp_name": "automl_sample"  # MLflow experiment name

}

'''The main FLAML AutoML API'''
with mlflow.start_run(nested=True, run_name="parallel_trial"):
    automl.fit(dataframe=join_customers, label='Churn', **settings)